# Singapore Road Data
This is a notebook that will guide you through 
1. Obtaining data from openstreetmaps
2. Generating useful output from the data
  - A visualisation of all the points
  - A text file with the coordinates, road names and number of lanes
  - A dumped dictionary and kd-tree, which can be used to quickly get information regarding a single point


In [12]:
# External imports
import overpy as op
import sys
import concurrent.futures
from tqdm import tqdm

# Internal imports (classes, functions)
from road_point import RoadPoint
from output_text import export_road_points
from output_scatter import make_gmap
from output_kd_tree import make_kdtree
from speed_limit_scraper import get_speed_limit

In [13]:
# Get API
api = op.Overpass()

# Query
query = "".join(l for l in open('query.xml'))
result = api.query(query)

Get all the ways (roads), do some duplicate removal and convert them to `RoadPoint`s. `RoadPoint` is a class used to encompass all the information we want about a coordinate.

In [14]:
# Get ways (roads or part of roads) 
# which have road names and lane count.
# Note that if we exclude the filters, there are many more 
# ponits to process (or even just lane count).
all_ways = result.get_ways()
ways = [w for w in all_ways if 'name' in w.tags and 'lanes' in w.tags]

# Remove duplicates in nodes (e.g 2 connected ways sharing a node)
node_dict = {}
for way in ways:
    for node in way.nodes:
        node_dict[node.id] = node

# Converts a node to a RoadPoint, without speed_limit
# This is done to prevent overloading in the multiprocessing
# step with the large overpy node object.
def node_to_RoadPoint(node):
    lat = float(node.lat)
    lon = float(node.lon)
    road_name = way.tags.get('name')
    lane_count =  way.tags.get('lanes')
    return RoadPoint(node.id, lat, lon, road_name, lane_count) 

unique_nodes = list(node_dict.values())
road_points_without_speedlimit = [node_to_RoadPoint(n) for n in unique_nodes]

Now, we use concurrency to add the speed limit into all the coordinates. We do this because fetching each speed limit takes a while, and it will be hard to process this for all the points sequentially.

**Proceeed to the next step once the success statement prints**

In [ ]:
# Variables used to show the progress bar
pbar = tqdm(total=len(road_points_without_speedlimit))


# Process a road point and add a speed limit property.
# Abstracted to enable concurrency for processing nodes.
def append_speed_limit(road_point):
    road_point.speed_limit = get_speed_limit(road_point.road_name)
    pbar.update(1)
    return road_point

    

executor = concurrent.futures.ThreadPoolExecutor()
print("Starting to scrape speed limits!")
road_points = list(executor.map(append_speed_limit, road_points_without_speedlimit))
print("Success!")
pbar.close()


  0%|          | 0/2880 [00:00<?, ?it/s]

Starting to scrape speed limits!



  6%|▌         | 161/2880 [00:14<04:09, 10.90it/s]


 11%|█         | 306/2880 [00:44<06:28,  6.63it/s]


 16%|█▌        | 466/2880 [01:00<13:04,  3.08it/s]


 21%|██        | 611/2880 [01:15<17:35,  2.15it/s]


 27%|██▋       | 773/2880 [01:31<03:28, 10.09it/s]


 32%|███▏      | 917/2880 [01:46<04:42,  6.95it/s]


 33%|███▎      | 959/2880 [01:48<06:27,  4.96it/s]

Now, we export all the points, in the different formats.

In [ ]:
# Text file (coordinates.txt)
export_road_points(road_points)
# Dictionary and KD tree dump (kdtree.txt and coords_to_roadpoints.txt)
kdtree = make_kdtree(road_points)
# Text file (my_map.html)
make_gmap(road_points)
